## Get the data

We can get the data using the entitysdk package. You will have to inform the id of the morphology you want to analyse.

To download the files, execute the following cell, and click on the link to set up authentication with the platform. If the code does not work, you might have to update the morphology_id, since it can change after a new deployment of the database.


In [11]:
import pandas
import neurom
import os
import numpy
from pathlib import Path

from ipywidgets import widgets, interact

import neurom.features

from obi_auth import get_token
from entitysdk.client import Client
from entitysdk.downloaders.morphology import download_morphology
from entitysdk.models.morphology import ReconstructionMorphology


token = get_token(environment="staging", auth_mode="daf")
client = Client(environment="staging", token_manager=token)

# morphology_id = "4ffcdcd4-f128-43c7-a657-4a925e18d9d8"  # Put your morphology id here
morphology_id = "bafe4dcb-ed75-44fc-a171-a41490b2c096"

# morph_tmp = client.search_entity(
#     entity_type=ReconstructionMorphology, query={"name": "rp190117_B_idB"}, limit=1
# ).one()
# print(morph_tmp.id)

morphology_entity = client.get_entity(entity_id=morphology_id, entity_type=ReconstructionMorphology)

def load_morphology(morph_path):
    try:
        m = neurom.load_morphology(morph_path)
    except Exception as e:
        print("Trouble with " + morph_path + ": " + str(e))
        return None
    return m


extension_wdgt = widgets.Dropdown(options=["asc", "swc"], description="Choose format to load")
display(extension_wdgt)


Dropdown(description='Choose format to load', options=('asc', 'swc'), value='asc')

In [12]:
output_dir = "./morphologies"
Path(output_dir).mkdir(parents=True, exist_ok=True)

morph_path = download_morphology(
    client,
    morphology_entity,
    output_dir = output_dir,
    file_type=extension_wdgt.value,
)
print(morph_path)
morph = load_morphology(str(morph_path))
_str = """Loaded morphology with {0} neurites of a total length of {1} um."""
display(_str.format(
    neurom.features.get("number_of_neurites", morph),
    numpy.sum(neurom.features.get("total_length_per_neurite", morph))
))

morphologies/C060114A5.swc


'Loaded morphology with 11 neurites of a total length of 21728.71990275383 um.'

We calculate for each segment of each neurite its path distance from the soma and its length.

The result is put into a handy DataFrame. For the calculations we use the neurom package.

In [13]:
res = pandas.concat([
    pandas.DataFrame({"lengths": neurom.features.neurite.segment_lengths(_nrt),
                    "pd": neurom.features.neurite.segment_path_lengths(_nrt),
                    "index": i, "type": _nrt.type
                    })
    for i, _nrt in enumerate(morph.neurites)
], axis=0)

display(res.head())

,lengths,pd,index,type
0,0.000000,0.000000,0,NeuriteType.axon
1,2.015146,2.015146,0,NeuriteType.axon
2,1.660789,3.675935,0,NeuriteType.axon
3,1.565399,5.241334,0,NeuriteType.axon
4,2.077421,7.318755,0,NeuriteType.axon


In [14]:
def calculate_results(vld_types, vld_range, show=True):
    msg = "You selected to measure the length of {0}\nbetween {1} and {2} um from the soma."
    if show:
        print(msg.format(vld_types, vld_range[0], vld_range[1]))

    fltr = (res["type"].isin(list(vld_types))) &\
    (res["pd"] >= vld_range[0]) & (res["pd"] < vld_range[1])
    ttl_len = res.loc[fltr]["lengths"].sum()
    if show:
        print("\nThe result in um is shown below:")
    return ttl_len


sel_types = widgets.SelectMultiple(options=res["type"].drop_duplicates(),
                                    index=[],
                                    description="Neurite types")
sel_range = widgets.IntRangeSlider(min=0, max=2000, step=1, description="Select range")
interact(calculate_results, vld_types=sel_types, vld_range=sel_range)

interactive(children=(SelectMultiple(description='Neurite types', options=(<NeuriteType.axon: 2>, <NeuriteType…

<function __main__.calculate_results(vld_types, vld_range, show=True)>

While the above is nice for interactive exploration, the below assigns results to variables that can be used for further analyses.

In [15]:
cutoff_proximal = (0, 100)
cutoff_distal = (100, 200)
used_neurite_types = [neurom.NeuriteType.basal_dendrite, neurom.NeuriteType.apical_dendrite]
tgt_total_count = 200

len_proximal = calculate_results(used_neurite_types, cutoff_proximal, show=False)
len_distal = calculate_results(used_neurite_types, cutoff_distal, show=False)

print("The ratio of proximal over distal dendrite length is {0}".format(
    len_proximal / len_distal
))

req_for_tgt_prox = tgt_total_count / len_proximal
req_for_tgt_dist = tgt_total_count / len_distal

print("""
{0} synapses require a mean length density of {1} on proximal dendrites.
{0} synapses require a mean length density of {2} on distal dendrites.
""".format(tgt_total_count, req_for_tgt_prox, req_for_tgt_dist))


The ratio of proximal over distal dendrite length is 1.6349904889593265

200 synapses require a mean length density of 0.060865696326846304 on proximal dendrites.
200 synapses require a mean length density of 0.09951483459828032 on distal dendrites.

